## Web Search Test
The **Web Search Test** ant demonstrates the use of a Web API secured using an API key with an auth header. This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.6.7"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAIConfigFactory.Get();
var client = GetOpenAIClient(config);


Installed Packages AntRunnerLib, 0.6.7

## Ensure the Web Search Test assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search Test" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search Test" contains the following files:
```
│   instructions.md - The instructions
│   manifest.json - The base assistant definition
│
└───OpenAPI
        api.bing.microsoft.com.json - A swagger file defining a tool named 'crawl' that uses web search
        auth.json - See below... 
```        

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [2]:
var assistantId = await AssistantUtility.Create("Web Search Test", config);
Console.WriteLine(assistantId)

asst_4EUSOW7dhpete4bUfLf1tVTG


## Run the Assistant

`output.Dialog` shows the conversation.

In [3]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search Test",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta this weekend. Be careful to not include events in the past in your final answer. You can perform multiple searches if necessary",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
Console.WriteLine(output.Dialog)


User: Use python to get todays date and then use it to find things to do in Atlanta this weekend. Be careful to not include events in the past in your final answer. You can perform multiple searches if necessary

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()

# Calculate the upcoming weekend dates (Saturday and Sunday)
# Assuming today is a weekday, we find the next Saturday and Sunday
days_until_saturday = (5 - today.weekday()) % 7
days_until_sunday = (6 - today.weekday()) % 7

# Get the dates for Saturday and Sunday
saturday = today + timedelta(days=days_until_saturday)
sunday = today + timedelta(days=days_until_sunday)

# Format the dates for output
saturday_date = saturday.strftime("%Y-%m-%d")
sunday_date = sunday.strftime("%Y-%m-%d")

today_date = today.strftime("%Y-%m-%d")
today_date, saturday_date, sunday_date
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on 2024-

In [4]:
Console.WriteLine(output.LastMessage)

Here are some events happening in Atlanta this weekend, September 14-15, 2024:

### Saturday, September 14, 2024:

1. **Atlanta Food and Wine Festival**
   - **Location:** Historic Fourth Ward Park
   - **Details:** Enjoy the best food, wine, beer, and cocktails Atlanta has to offer. This event brings in 150+ chefs, sommeliers, mixologists, and industry insiders for 5 days of mouthwatering and tempting cuisines & special Tasting Tents.
   - [More Information](https://www.experism.com/atlanta/september)

2. **The Princess Bride in Concert with the ASO**
   - **Location:** Atlanta Symphony Hall
   - **Details:** Experience the iconic film The Princess Bride in a new way, with a live orchestral performance by the Atlanta Symphony Orchestra.
   - [More Information](https://discoveratlanta.com/events/main/)

3. **Sweet Auburn Music Fest**
   - **Location:** Auburn Avenue
   - **Details:** The festival offers fun-filled entertainment, live music (everything from R&B to gospel to alternative 

## Clean Up

In [5]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Web Search Test");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
